<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/BERT/BERTimbau_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 41.3 MB/s 
     |████████████████████████████████| 895 kB 67.9 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.

     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 17.4 MB 1.3 MB/s 
     |████████████████████████████████| 1.9 MB 19.7 MB/s 
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.8.1, torchaudio==0.8.0 and torchvision==0.9.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==1.8.1
    torchvision 0.9.1 depends on torch==1.8.1
    torchaudio 0.8.0 depends on torch==1.8.0

To fix this you could try to:
1. loosen the range of package versions you'

In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [12]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from google.colab import files, drive
import io, os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)

# Preliminaries

from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

# Training

import torch.optim as optim

# Evaluation
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, precision_score, \
                            recall_score,  f1_score, confusion_matrix, \
                            plot_confusion_matrix, classification_report, \
                            balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef
import seaborn as sns

In [5]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks
source_folder = './amostra_news_integrada/amostra_' #'/content'
destination_folder = './bertimbau_resp' #'/content'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks


In [19]:
class BERTimbau_Noticias():

    def __init__(self, tokenizer, max_seq_length=512, 
               bs=8, lr=1e-6, qtd_categorias=10,
               source_folder = './amostra_news_integrada/amostra_', 
               destination_folder = './bertimbau_resp'):
      self.MAX_SEQ_LEN = max_seq_length #limita os artigos em 128 tokens. BERTimbau base é limitado em 512 tokens por texto.
      self.PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
      self.UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
      self.bs = bs
      self.lr = lr
      self.qtd_categorias=qtd_categorias
      self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
      self.source_folder = source_folder
      self.destination_folder= destination_folder
    

    def montaFields(self):
      '''Define a estrutura do registro que será utilizada para construir os datasets de treinamento, validação e teste. 
        Também faz a ligação entre o tokenizador BERTimbau e as sequências de texto. 
        Esta estrutura será submetida ao tokenizaAmostra.
        Retorna uma lista contendo o label_field, text_field e o fields que é uma lista de tuplas com text e label'''
      # Fields - use_vocab=False  e tokenizer.encode permite que utilizemos os tokens do BERTimbau.
      label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
      text_field = Field(use_vocab=False, tokenize=self.tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                    fix_length=self.MAX_SEQ_LEN, pad_token=self.PAD_INDEX, unk_token=self.UNK_INDEX)
      fields = [('text', text_field),('label', label_field)]
      return [label_field, text_field, fields]

    def tokenizaAmostra(self, source_folder, soTeste=False):
      ''' Constroi os dataset de treino, validação e teste a partir de csv independentes disponibilizados no caminho source_folder e utilizando
          a estrutura de registro definida em fields, transformando a sequência de textos em sequência de tokens.
          Depois monta os BuckIterator de treino, validação e teste conforme definição do tamanho do bach size.
          Retorna uma lista de Iterators: treino, validação e teste.
          '''    
      #Fields
      label_field, text_field, fields = montaFields()
      # TabularDataset
      train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
                                                test='test.csv', format='CSV', fields=fields, skip_header=True)
      # Iterators

      train_iter, valid_iter = None, None
      test_iter = Iterator(test, batch_size=self.bs, device=self.device, train=False, shuffle=False, sort=False)
      if soTeste == False:
        train_iter = BucketIterator(train, batch_size=self.bs, sort_key=lambda x: len(x.text),
                                  device=self.device, train=True, sort=True, sort_within_batch=True)
        valid_iter = BucketIterator(valid, batch_size=self.bs, sort_key=lambda x: len(x.text),
                                  device=self.device, train=True, sort=True, sort_within_batch=True)
      return [train_iter, valid_iter, test_iter]

    def save_checkpoint(self,save_path, model, valid_loss):

      if save_path == None:
          return
      
      state_dict = {'model_state_dict': model.state_dict(),
                    'valid_loss': valid_loss}
      
      torch.save(state_dict, save_path)
      print(f'Model saved to ==> {save_path}')

    def load_checkpoint(self,load_path, model):
      
      if load_path==None:
          return
      
      state_dict = torch.load(load_path, map_location=device)
      print(f'Model loaded from <== {load_path}')
      
      model.load_state_dict(state_dict['model_state_dict'])
      return state_dict['valid_loss']


    def save_metrics(self,save_path, train_loss_list, valid_loss_list, global_steps_list):

      if save_path == None:
          return
      
      state_dict = {'train_loss_list': train_loss_list,
                    'valid_loss_list': valid_loss_list,
                    'global_steps_list': global_steps_list}
      
      torch.save(state_dict, save_path)
      print(f'Model saved to ==> {save_path}')


    def load_metrics(self,load_path):

      if load_path==None:
          return
      
      state_dict = torch.load(load_path, map_location=device)
      print(f'Model loaded from <== {load_path}')
      
      return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

    def save_embedding(self,save_path, embedding):

      if save_path == None:
          return
      
      emb_dict = {'_': embedding}
      
      torch.save(emb_dict, save_path)
      print(f'Embedding saved to ==> {save_path}')

    def load_embedding(self,load_path):

      if load_path==None:
          return
      
      emb_dict = torch.load(load_path, map_location=device)
      print(f'Embedding loaded from <== {load_path}')
      
      return emb_dict

    def create_directory(self,path, directory):
      try:
        os.makedirs(path + '/' + directory)
      except FileExistsError:
        # directory already exists
        pass
      
    def train(self, model,
          optimizer,
          train_loader ,
          valid_loader ,
          eval_every ,
          file_path ,
          criterion = nn.CrossEntropyLoss(), #nn.BCELoss(),
          num_epochs = 5,
          best_valid_loss = float("Inf")):
    
    
      print('1 - inicializando variávies')
      # initialize running values
      running_loss = 0.0
      valid_running_loss = 0.0
      global_step = 0
      train_loss_list = []
      valid_loss_list = []
      global_steps_list = []

      # training loop
      print('1 - inicializando treinamento')
      model.train() #habilita os valores dos pesos do modelo para treinamento
      for epoch in range(num_epochs):
          X = []  
          for (text, labels), _ in train_loader:
              labels = labels.type(torch.LongTensor)  #carrega as categorias para pytorch         
              labels = labels.to(device) # disponibiliza para as GPU
              text = text.type(torch.LongTensor) #carrega os tokens do texto para pytorch        
              text = text.to(device) #disponibiliza para as GPU
              output = model(text, labels) #efetua o treinamento no modelo bert carregado
              loss, _ = output #obtem o valor da função loss do texto treinado

              optimizer.zero_grad() #limpa os gradientes do último treino (zera tudo)
              loss.backward() #calcula a derivada da função perda em relação aos parâmetros
              optimizer.step() #atualiza os pesos fazendo com que o otimizador dê um passo com base no gradiente dos parâmetros

              # update running values
              running_loss += loss.item() #acumula o valor da função loss
              global_step += 1
              #X.extend(_.tolist())
              # evaluation step - roda o modelo para o token de validação, obtém o valor da função loss e tira a média para o treinamento e validação.
              if global_step % eval_every == 0:
                  model.eval() # seta as camadas dropout e batch normalization 
                  with torch.no_grad():                    
                    
                      # validation loop
                      for (text,labels), _ in valid_loader:
                          text = text.type(torch.LongTensor)  
                          text = text.to(device)
                          labels = labels.type(torch.LongTensor)           
                          labels = labels.to(device)
                          
                          output = model(text, labels)
                          loss, _ = output
                          
                          valid_running_loss += loss.item()
                          
                  
                      

                  # evaluation
                  average_train_loss = running_loss / eval_every
                  average_valid_loss = valid_running_loss / len(valid_loader)
                  train_loss_list.append(average_train_loss)
                  valid_loss_list.append(average_valid_loss)
                  global_steps_list.append(global_step)

                  # resetting running values
                  running_loss = 0.0                
                  valid_running_loss = 0.0
                  model.train()

                  # print progress
                  print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                        .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                                average_train_loss, average_valid_loss))
                  
                  # checkpoint
                  if best_valid_loss > average_valid_loss:
                      best_valid_loss = average_valid_loss
                      self.save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                      self.save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
                      
          #print(X)
          self.save_embedding(file_path + '/' + 'embedd.pt', X)
      self.save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
      #save_embedding(file_path + '/' + 'embedd.pt', X)
      print('Finished Training!')

    def calcula_GMean_multiclass(self,revocacao):
      revoc = np.array(revocacao)
      GMean = revoc.prod()**(1.0/len(revoc))
      return GMean 

    def calcula_especificidade(self, matriz_confusao):
      FP = matriz_confusao.sum(axis=0) - np.diag(matriz_confusao) 
      FN = matriz_confusao.sum(axis=1) - np.diag(matriz_confusao)
      VP = np.diag(matriz_confusao)
      VN = matriz_confusao.sum() - (FP + FN + VP)
      FP = FP.astype(float)
      FN = FN.astype(float)
      VP = VP.astype(float)
      VN = VN.astype(float)
      TVN = np.sum(VN)/(np.sum(VN) + np.sum(FP))
      return TVN
    
    def calcula_especificidade_porClasse(self, matriz_confusao):
      FP = matriz_confusao.sum(axis=0) - np.diag(matriz_confusao) 
      FN = matriz_confusao.sum(axis=1) - np.diag(matriz_confusao)
      VP = np.diag(matriz_confusao)
      VN = matriz_confusao.sum() - (FP + FN + VP)
      FP = FP.astype(float)
      FN = FN.astype(float)
      VP = VP.astype(float)
      VN = VN.astype(float)
      TVN = (VN)/(VN + FP)
      return TVN

    def elabora_relatorio_metricas(self, report, matriz_confusao):
      espec = calcula_especificidade_porClasse(matriz_confusao) # calcula o valor da especificidade para cada classe
      dfrep = pd.DataFrame(report).transpose() #transforma o conteúdo do classification_report em um dataframe pandas
      dfrep_a = dfrep[:-3].copy() # separa as métricas de cada classe do valor da acurácia geral do modelo
      dfrep_a['specificity'] = espec # inclui no dataframe o valor da especificidade
      dfrep_b  = dfrep[dfrep.index=='accuracy'].copy() #obtem do dataframe somente o valor da acurácia
      dfrep_b['specificity']=calcula_especificidade(matriz_confusao) # inclui o valor da especificidade geral de todas as classes
      metricas = ['precision', 'recall', 'specificity', 'f1-score', 'support'] #organiza as métricas na ordem desejada
      df = pd.concat([dfrep_a[metricas],dfrep_b[metricas]],sort=False) #concatena todos os valores em um único dataframe
      df['support'] = df['support'].astype('int')
      return df

    def elabora_relatorio_metricas(self, report, matriz_confusao):
      espec = calcula_especificidade_porClasse(matriz_confusao) # calcula o valor da especificidade para cada classe
      dfrep = pd.DataFrame(report).transpose() #transforma o conteúdo do classification_report em um dataframe pandas
      dfrep_a = dfrep[:-3].copy() # separa as métricas de cada classe do valor da acurácia geral do modelo
      dfrep_a['specificity'] = espec # inclui no dataframe o valor da especificidade
      dfrep_b  = dfrep[dfrep.index=='accuracy'].copy() #obtem do dataframe somente o valor da acurácia
      dfrep_b['specificity']=calcula_especificidade(matriz_confusao) # inclui o valor da especificidade geral de todas as classes
      metricas = ['precision', 'recall', 'specificity', 'f1-score', 'support'] #organiza as métricas na ordem desejada
      df = pd.concat([dfrep_a[metricas],dfrep_b[metricas]],sort=False) #concatena todos os valores em um único dataframe
      df['support'] = df['support'].astype('int')
      return df

    def evaluate(self, model, test_loader):
      y_pred = []
      y_true = []
      X = []

      #obtem os valores preditos e os valores de teste
      model.eval()
      with torch.no_grad():
          for (text, labels), _ in test_loader:
                  labels = labels.type(torch.LongTensor)  #carrega as classes para uma estrutura pytorch         
                  labels = labels.to(device)  #carrega a estrutura pytorch para GPU (se houver, foi testado anteriormente)
                  text = text.type(torch.LongTensor)    #carrega o texto para uma estrutura pytorch
                  text = text.to(device)          #carrega a estrutura pytorch para GPU (se houver, foi testado anteriormente)
                  output = model(text, labels)   #submete o texto e a label da classe ao modelo

                  _, output = output
                  y_pred.extend(torch.argmax(output, 1).tolist())  #obtem do pytorch de saída do modelo o valor predito.
                  y_true.extend(labels.tolist()) #obtem do pytorch de teste  valor real.
                  #print(_)
                  X.extend(output.tolist())
      
      return [X, y_true, y_pred]

    def obtemMetricas(self, y_true, y_pred, ):
      n_classe = np.max(y_true)+1 #obtem o número de classes
      acuraccy = accuracy_score(y_true, y_pred)
      bal_accuracy = balanced_accuracy_score(y_true, y_pred)
      precision_global = precision_score(y_true, y_pred, average='micro')
      precision_local = precision_score(y_true, y_pred, average='macro')
      recall_global = recall_score(y_true, y_pred, average='micro')
      recall_local = recall_score(y_true, y_pred, average='macro')
      f1score_global = f1_score(y_true, y_pred, average='micro')
      f1score_local = f1_score(y_true, y_pred, average='macro')
      cohen_kappa = cohen_kappa_score(y_true, y_pred,labels=np.arange(0,n_classe))
      ccmatheus = matthews_corrcoef(y_true, y_pred)
      return[acuraccy, bal_accuracy, precision_global, recall_global, f1score_global,
             cohen_kappa, ccmatheus, precision_local, recall_local, f1score_local ]


    
    def imprimeMetricas(self, y_true, y_pred):
      print('Classification Report:')
      n_classe = np.max(y_true)+1 #obtem o número de classes
      report = classification_report(y_true, y_pred, labels=np.arange(0,n_classe), digits=4, output_dict=True) #gera o relatório de métricas
      cm = confusion_matrix(y_true, y_pred, labels=np.arange(0,n_classe)) #gera a matriz de confusao
      report = elabora_relatorio_metricas(report, cm) #inclui no relatorio a especificidade
      print(report)

      cohen_kappa = cohen_kappa_score(y_true, y_pred,labels=np.arange(0,n_classe))
      ccmatheus = matthews_corrcoef(y_true, y_pred)
      
      print("Acurácia:",accuracy_score(y_true, y_pred))
      print('Acurácia balanceada:',balanced_accuracy_score(y_true, y_pred))
      print('GMean:', calcula_GMean_multiclass(report['recall']))
      print('Cohen Kappa Score:', cohen_kappa)
      print('Coef. Correlacao Matheus:', ccmatheus)
      print('******************************************************************')
      
      
      #ax= plt.subplot()
      #sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")
      #ax.set_title('Confusion Matrix')
      #ax.set_xlabel('Predicted Labels')
      #ax.set_ylabel('True Labels')
      return [X, y_true, y_pred, report]


class BERT(nn.Module):

    def __init__(self, qtd_categories):
        super(BERT, self).__init__()

        options_name = "bert-base-portuguese-cased"
        self.encoder = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased',num_labels=qtd_categories )

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]
        
        return loss, text_fea

In [ ]:
processo = BERTimbau_Noticias(tokenizer = tokenizer,
                              max_seq_length = 512,
                              bs=8,
                              lr=1e-06,
                              qtd_categorias=10)

%%time
#token_train, token_valid, token_test = [],[],[]
id_planilha = 9.4
dataset = 5
n_epoca = 5
for idx in range(10):
  processo.create_directory(processo.destination_folder,str(id_planilha)+'/'+str(idx))
  file_path = destination_folder + "/"+str(id_)+'/'+str(idx)
  model = BERT(qtd_categories=processo.qtd_categorias).to(processo.device)
  optimizer = optim.Adam(model.parameters(), lr=processo.lr)
  inicio = datetime.now()
  print('Treinando amostra:', idx, 'inicio:', inicio)
  tokens = processo.tokenizaAmostra(processo.source_folder+str(dataset))
  train(model=model, 
      optimizer=optimizer,
      num_epochs=n_epoca,
      train_loader=tokens[0],
      valid_loader=tokens[1],
      eval_every=len(tokens[0]) // 2, #tamanho do dataset de treinamento / 2 (chão)
      file_path=file_path)
  print('Duracao:', datetime.now()-inicio)